In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import random
from tqdm.notebook import tqdm
np.random.seed(1)

In [ ]:
pneumonia_paths = glob.glob('/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/*.jpeg', recursive = True)
normal_paths = glob.glob('/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/*.jpeg', recursive = True)

In [ ]:
def resize_image_square_opencv(image_path, target_size):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (target_size, target_size))
    return resized_image

In [ ]:
import numpy as np
import pandas as pd
from keras.layers import Conv2D, Dense, MaxPooling2D, Softmax, Dropout, BatchNormalization, Flatten
from keras import Sequential
import keras
import tensorflow as tf
import cv2 as cv
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras import optimizers

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape = (128,128,1), activation="relu", filters= 64, strides=1, kernel_size= (8,8)))
model.add(MaxPooling2D(pool_size= (8,8)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(1,"softmax"))

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer = "RMSprop",metrics=["accuracy"])
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=2)

In [ ]:
#resnet model
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(180,180,3),
                   pooling='avg',classes=5,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(1, activation='softmax'))
model = resnet_model

adam = optimizers.Adam()
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)

In [ ]:
X = []
Y = []

In [ ]:
for i in tqdm(pneumonia_paths[:1341]):
    X.append(resize_image_square_opencv(i, 180))
    Y.append(1)

for i in tqdm(normal_paths):
    X.append(resize_image_square_opencv(i, 180))
    Y.append(0)

0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [ ]:
X = np.array(X)
Y = np.array(Y)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint_acc = ModelCheckpoint("best_raw.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
checkpoint_val = ModelCheckpoint("best_raw.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=False)

In [ ]:
history = model.fit(X_train, Y_train,epochs=50,batch_size=32, verbose=1, validation_split = 0.2, callbacks = [checkpoint_val, checkpoint_acc]  )


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
results  = model.evaluate(X_test, Y_test, batch_size = 2)

In [ ]:
print("test loss : ", results[0], " test accuracy : ", results[1])